<a href="https://colab.research.google.com/github/diogon01/42robotics/blob/master/notebooks/trabalhoV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Clonar o repositório do GitHub no ambiente do Colab
!git clone --branch feature/entrega-final https://github.com/felipeabe/artificial-neural-network.git

# Adicionar o diretório src ao sys.path para permitir a importação de módulos
import sys
sys.path.append('/content/artificial-neural-network/src')

# Verificar se o repositório foi clonado corretamente
!ls /content/artificial-neural-network


Cloning into 'artificial-neural-network'...
remote: Enumerating objects: 112, done.
remote: Counting objects: 100% (112/112), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 112 (delta 48), reused 27 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (112/112), 901.24 KiB | 4.98 MiB/s, done.
Resolving deltas: 100% (48/48), done.
datasets  notebooks  README.md	src  teste.ipynb


In [3]:
# Importando bibliotecas necessárias
import pandas as pd

# Importando o dataset 'obesity.csv'
# Este conjunto contém dados sobre obesidade com fatores como estilo de vida e transporte
dados_obesidade = pd.read_csv("/content/artificial-neural-network/datasets/obesity.csv")

# Renomear colunas para nomes mais intuitivos
mapear_colunas = {
    'family_history_with_overweight': 'historico_familiar',
    'FAVC': 'comida_calorica',
    'FCVC': 'vegetais',
    'NCP': 'refeicoes_dia',
    'SMOKE': 'fuma',
    'CH2O': 'agua_dia',
    'SCC': 'calorias',
    'FAF': 'atividade_fisica',
    'CALC': 'alcool',
    'MTRANS': 'transporte',
    'NObeyesdad': 'nivel_obesidade'
}
dados_obesidade = dados_obesidade.rename(columns=mapear_colunas)

# Remover colunas irrelevantes para a análise
colunas_remover = ['CAEC', 'TUE']
dados_obesidade = dados_obesidade.drop(columns=colunas_remover)

# Codificar variáveis categóricas (binárias e escalares)
mapeamento_binario = {
    'Gender': {"Female": 0, "Male": 1},
    'historico_familiar': {"yes": 1, "no": 0},
    'comida_calorica': {"yes": 1, "no": 0},
    'fuma': {"yes": 1, "no": 0},
    'calorias': {"yes": 1, "no": 0}
}
for coluna, mapeamento in mapeamento_binario.items():
    dados_obesidade[coluna] = dados_obesidade[coluna].map(mapeamento)

# Codificar consumo de álcool e transporte
mapeamento_alcool = {'no': 0, 'Sometimes': 1, 'Frequently': 2, 'Always': 3}
dados_obesidade['alcool'] = dados_obesidade['alcool'].map(mapeamento_alcool)

mapeamento_transporte = {
    'Walking': 0, 'Bike': 0,
    'Motorbike': 1, 'Automobile': 1,
    'Public_Transportation': 1
}
dados_obesidade['transporte'] = dados_obesidade['transporte'].map(mapeamento_transporte)

# Normalizar dados contínuos (altura, peso, idade)
colunas_normalizar = ['Height', 'Weight', 'Age']
dados_obesidade[colunas_normalizar] = dados_obesidade[colunas_normalizar].apply(
    lambda x: ((x - x.min()) / (x.max() - x.min())).round(3)
)

# Dividir o dataset em treino (80%) e teste (20%)
split_point = int(0.8 * len(dados_obesidade))
treino_multiclasse = dados_obesidade[:split_point]
teste_multiclasse = dados_obesidade[split_point:]


In [5]:
# Importando bibliotecas necessárias
import pandas as pd

# Importar o dataset 'houses.csv'
# Este conjunto contém dados sobre propriedades como preço e área útil
dados_imoveis = pd.read_csv("/content/artificial-neural-network/datasets/houses.csv")

# Remover colunas irrelevantes
# As colunas 'renovated' (renovação) e 'quartile_zone' (zona por quartil) são removidas por serem irrelevantes
colunas_remover = ['renovated', 'quartile_zone']
dados_imoveis = dados_imoveis.drop(columns=colunas_remover)

# Codificar valores booleanos (True/False) em numéricos (1/0)
# Transformação necessária para modelos de aprendizado que lidam com dados numéricos
dados_imoveis = dados_imoveis.replace({True: 1, False: 0})

# Extrair o ano da coluna de data
# Converte a coluna 'date' para o formato datetime e extrai apenas o ano para análise
dados_imoveis['date'] = pd.to_datetime(dados_imoveis['date'])
dados_imoveis['ano_construcao'] = dados_imoveis['date'].dt.year
dados_imoveis = dados_imoveis.drop(columns=['date'])  # Remove a coluna original de data

# Normalizar colunas contínuas ('price' e 'living_in_m2') para valores entre 0 e 1
colunas_normalizar = ['price', 'living_in_m2']
dados_imoveis[colunas_normalizar] = dados_imoveis[colunas_normalizar].apply(
    lambda x: ((x - x.min()) / (x.max() - x.min())).round(3)
)

# Dividir o conjunto de dados em treino (80%) e teste (20%)
split_point = int(0.8 * len(dados_imoveis))
treino_regressao = dados_imoveis[:split_point]  # Dados para treino
teste_regressao = dados_imoveis[split_point:]  # Dados para teste

# Exibir as 10 primeiras linhas do conjunto de treino para validação
print(treino_regressao.head(10))


   price  bedrooms  grade  has_basement  living_in_m2  nice_view  \
0  0.215         2      2             1         0.261          0   
1  0.213         2      2             0         0.166          0   
2  0.223         2      2             0         0.201          1   
3  0.171         2      3             0         0.299          0   
4  0.576         3      2             1         0.856          0   
5  0.339         1      2             1         0.103          1   
6  0.149         2      2             1         0.253          0   
7  0.143         1      1             0         0.264          0   
8  0.116         2      2             0         0.166          0   
9  0.432         2      3             0         0.204          0   

   perfect_condition  real_bathrooms  has_lavatory  single_floor  month  \
0                  0               2             1             0      5   
1                  0               1             1             0     11   
2                  0      

<ipython-input-5-1a7a20d52bb9>:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dados_imoveis = dados_imoveis.replace({True: 1, False: 0})


In [8]:
# Importar bibliotecas necessárias
# Utilizamos pandas para manipulação de dados e numpy para operações matemáticas
import pandas as pd
import numpy as np

# Importar o dataset 'alzheimer.csv'
# Este conjunto contém informações sobre pacientes e diagnósticos
dados_alzheimer = pd.read_csv("/content/artificial-neural-network/datasets/alzheimer.csv")

# Visualizar as primeiras linhas do conjunto de dados
# Isso ajuda a verificar se o dataset foi carregado corretamente
print(dados_alzheimer.head())

# Remover colunas irrelevantes
# As colunas 'PatientID' e 'DoctorInCharge' são consideradas irrelevantes para o diagnóstico
dados_alzheimer = dados_alzheimer.drop(columns=["PatientID", "DoctorInCharge"])

# Separar variáveis categóricas e numéricas
# Variáveis categóricas como gênero, etnia e nível educacional são processadas separadamente
variaveis_categoricas = ["Gender", "Ethnicity", "EducationLevel"]
variaveis_numericas = [col for col in dados_alzheimer.columns if col not in variaveis_categoricas + ["Diagnosis"]]

# Codificar variáveis categóricas
# OneHotEncoder é usado para transformar categorias em representações numéricas binárias
from sklearn.preprocessing import OneHotEncoder
codificador = OneHotEncoder(drop="first", sparse_output=False)  # Evitar multicolinearidade
categorias_codificadas = codificador.fit_transform(dados_alzheimer[variaveis_categoricas])
categorias_codificadas_df = pd.DataFrame(categorias_codificadas, columns=codificador.get_feature_names_out(variaveis_categoricas))

# Normalizar variáveis numéricas
# As variáveis numéricas são escaladas para manter uma escala consistente (média 0, desvio padrão 1)
from sklearn.preprocessing import StandardScaler
escalador = StandardScaler()
numericas_normalizadas = escalador.fit_transform(dados_alzheimer[variaveis_numericas])
numericas_normalizadas_df = pd.DataFrame(numericas_normalizadas, columns=variaveis_numericas)

# Combinar dados processados com a variável alvo
# As variáveis categóricas codificadas e as numéricas normalizadas são combinadas em um único conjunto
dados_processados = pd.concat([numericas_normalizadas_df, categorias_codificadas_df], axis=1)
dados_processados["Diagnosis"] = dados_alzheimer["Diagnosis"]

# Dividir os dados em conjuntos de treino e teste
# Usamos 80% dos dados para treino e 20% para teste
from sklearn.model_selection import train_test_split
X = dados_processados.drop(columns=["Diagnosis"])  # Dados de entrada
y = dados_processados["Diagnosis"]  # Variável alvo (diagnóstico)
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=42)

# Exibir as dimensões dos conjuntos de treino e teste
print(f"Conjunto de treino: {X_treino.shape}")
print(f"Conjunto de teste: {X_teste.shape}")


   PatientID  Age  Gender  Ethnicity  EducationLevel        BMI  Smoking  \
0       4751   73       0          0               2  22.927749        0   
1       4752   89       0          0               0  26.827681        0   
2       4753   73       0          3               1  17.795882        0   
3       4754   74       1          0               1  33.800817        1   
4       4755   89       0          0               0  20.716974        0   

   AlcoholConsumption  PhysicalActivity  DietQuality  ...  MemoryComplaints  \
0           13.297218          6.327112     1.347214  ...                 0   
1            4.542524          7.619885     0.518767  ...                 0   
2           19.555085          7.844988     1.826335  ...                 0   
3           12.209266          8.428001     7.435604  ...                 0   
4           18.454356          6.310461     0.795498  ...                 0   

   BehavioralProblems       ADL  Confusion  Disorientation  \
0     

In [12]:
# Adicionar o diretório `src` ao caminho do Python
import sys
sys.path.append('/content/artificial-neural-network/src')

# Importar a implementação da Rede Neural
from neural_network import NeuralNetwork
from activations import relu, sigmoid, tanh
from losses import mse_loss, cross_entropy_loss

# Configurar a Rede Neural
configuracao = {
    "input_size": X_treino.shape[1],  # Número de características (colunas) no conjunto de treino
    "hidden_size": 10,  # Número de neurônios na camada oculta
    "output_size": 1,  # Saída com 1 neurônio (para classificação binária)
    "activation": "relu",  # Função de ativação escolhida
    "loss": "mse",  # Função de perda escolhida
    "learning_rate": 0.01,  # Taxa de aprendizado para o gradiente descendente
    "epochs": 1000  # Número de épocas de treinamento
}

# Instanciar a Rede Neural
rede_neural = NeuralNetwork(
    input_size=configuracao["input_size"],
    hidden_size=configuracao["hidden_size"],
    output_size=configuracao["output_size"],  # Adicionado output_size
    activation=configuracao["activation"],
    loss=configuracao["loss"]
)

# Treinar a Rede Neural com os dados de treino
# Verificar se o método correto é 'fit', 'train' ou outro equivalente
rede_neural.fit(  # Substitua 'fit' pelo método correto implementado na classe
    X_treino.to_numpy(),
    y_treino.to_numpy().reshape(-1, 1),
    configuracao["learning_rate"],
    configuracao["epochs"]
)

# Avaliar a Rede Neural com os dados de teste
acuracia = rede_neural.evaluate(  # Substitua 'evaluate' pelo método correto implementado na classe
    X_teste.to_numpy(),
    y_teste.to_numpy().reshape(-1, 1)
)

# Exibir o resultado da acurácia
print(f"Acurácia do modelo: {acuracia * 100:.2f}%")


TypeError: 'float' object cannot be interpreted as an integer